# Walmart Sales: Pre-processing data before modeling 

In this notebook I  cleaned and processed the Wallmart sales data [(Kaggle competition)](https://www.kaggle.com/competitions/walmart-sales-forecasting/overview) before using it in a machile learning model setting.  
 
 In more detail, I:  
- Separated the **target** from the **explanatory** **variables**
- **Feature Engineering**: Created new categorical variables from continuous explanatory variables


## Table of Contents  
1. Target variable and explanatory variables 
2. Explanatory variables: feature engineering  
3. Export processed variables for model training  
4. Data processing summary

## Import libraries

In [95]:
import pandas as pd
import numpy as np

# Import my own functions and/or variables
from src.features.build_features import store_cat_fn, quarter_dic, year_dic, temp_bin_fn, usa_hols_df


## Import data

In [96]:
# Import eda pre-processed data
data_eda_df = pd.read_csv("data/interim/Walmart_Store_sales.csv").iloc[: , 1:] # drop first unnamed index column
print("There are {} samples and {} variables present in the EDA processed data".format(data_eda_df.shape[0], data_eda_df.shape[1]))
data_eda_df.head()


There are 136 samples and 18 variables present in the EDA processed data


,Store,date,dayofweek,dayofweek_en,quarter,month,month_en,year,dayofyear,dayofmonth,weekofyear,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Store_str
0,6.0,2011-02-18,4.0,Friday,1.0,2.0,Feb,2011.0,49.0,18.0,7.0,1572117.54,NaN,15.338889,3.045,214.777523,6.858,6.0
1,13.0,2011-03-25,4.0,Friday,1.0,3.0,Mar,2011.0,84.0,25.0,12.0,1807545.43,0.0,5.766667,3.435,128.616064,7.470,13.0
2,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1244390.03,0.0,29.205556,NaN,214.556497,7.346,11.0
3,6.0,2010-05-28,4.0,Friday,2.0,5.0,May,2010.0,148.0,28.0,21.0,1644470.66,0.0,26.050000,2.759,212.412888,7.092,6.0
4,4.0,2010-05-28,4.0,Friday,2.0,5.0,May,2010.0,148.0,28.0,21.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,4.0


In [97]:
# Variable number and types
data_eda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         136 non-null    float64
 1   date          118 non-null    object 
 2   dayofweek     118 non-null    float64
 3   dayofweek_en  118 non-null    object 
 4   quarter       118 non-null    float64
 5   month         118 non-null    float64
 6   month_en      118 non-null    object 
 7   year          118 non-null    float64
 8   dayofyear     118 non-null    float64
 9   dayofmonth    118 non-null    float64
 10  weekofyear    118 non-null    float64
 11  Weekly_Sales  136 non-null    float64
 12  Holiday_Flag  125 non-null    float64
 13  Temperature   121 non-null    float64
 14  Fuel_Price    124 non-null    float64
 15  CPI           125 non-null    float64
 16  Unemployment  122 non-null    float64
 17  Store_str     136 non-null    float64
dtypes: float64(15), object(3)
memo

## Drop outliers  

Based on EDA, drop store 12

In [98]:
# Create copy of original data
data_model_df = data_eda_df.copy()

data_model_df = data_model_df[data_model_df['Store'] !=12] #Note: tried dropping this sample but performance of model was worse, so kept it with a tag

print("There are {} samples and {} variables present".format(data_model_df.shape[0], data_model_df.shape[1]))

There are 131 samples and 18 variables present


## 1. Target variable and explanatory variables

In [99]:
# Store_str imported as float, re-convert to string and drop Store column
data_model_df['Store_str'] = [str(store) for store in data_model_df.Store.tolist()]

## 1a. Target Variable

In [100]:
target_variable = "Weekly_Sales"
Y = data_model_df.loc[:,target_variable]
print(Y.shape)
print(Y.head())

(131,)
0    1572117.54
1    1807545.43
2    1244390.03
3    1644470.66
4    1857533.70
Name: Weekly_Sales, dtype: float64


## 1b. Explanatory Variables

In [101]:
# Drop  date variables which are collinear (see 01-EDA.ipynb for correlation heatmap)
# Keep store ID and Weekly sales for feature engineering
features_drop_ls= ['Store', 'date', 'dayofweek', 'dayofweek_en', 'month', 'month_en','dayofyear', 'dayofmonth']

print('Shape before filter:', data_model_df.shape)
X_raw = data_model_df.drop(columns=features_drop_ls, axis=1)
print('Shape of explanatory variables dataframe after filter:', X_raw.shape)


Shape before filter: (131, 18)
Shape of explanatory variables dataframe after filter: (131, 10)


### Convert year variable to string

In [102]:
X_raw['year'] = X_raw['year'].map(year_dic)
# Check conversion
type(X_raw.year.tolist()[0])

str

In [103]:
# Description of variables
X_raw.describe(include='all')

,quarter,year,weekofyear,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Store_str
count,113.000000,113,113.000000,1.310000e+02,120.000000,117.000000,119.000000,120.000000,117.000000,131
unique,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
top,NaN,y1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
freq,NaN,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
mean,2.415929,NaN,25.159292,1.257990e+06,0.066667,15.781054,3.302908,180.175755,7.399427,NaN
std,1.066751,NaN,13.818056,6.577463e+05,0.250490,10.259300,0.475435,39.723167,0.994117,NaN
min,1.000000,NaN,1.000000,2.689290e+05,0.000000,-7.338889,2.514000,126.111903,5.143000,NaN
25%,2.000000,NaN,15.000000,5.842439e+05,0.000000,7.122222,2.824000,132.579257,6.664000,NaN
50%,2.000000,NaN,25.000000,1.366396e+06,0.000000,16.550000,3.435000,197.655672,7.368000,NaN
75%,3.000000,NaN,36.000000,1.809576e+06,0.000000,24.188889,3.708500,214.904838,8.099000,NaN


## 2. Explanatory variables: feature engineering

### Create groups of stores based on weekly sales, CPI and/or unemployment rate

In [104]:
# Create categories for store based on EDA figure comparing Weekly Sales vs CPI and/or Unemployment rate:

# Define thresholds to be used for categories
median_weekly_sales = round(X_raw['Weekly_Sales'].median(), 2)
high_CPI = 150
median_unemp = round(X_raw['Unemployment'].median(), 2)

# Create categories and column. Define store 12 as outlier sample (cf EDA plot CPI vs Unemployment and weekly sales in section 4)
X_feateng = store_cat_fn(data=X_raw, thresh_sales=median_weekly_sales, thresh_CPI=high_CPI, thresh_unemp=median_unemp, outlier_samp='12.0')

print('Data shape before feat eng:', X_raw.shape)
print('Data shape after feat eng:', X_feateng.shape)


Data shape before feat eng: (131, 10)
Data shape after feat eng: (131, 12)


In [105]:
# Describe newly created store groups
print('Store groups based on weekly sales and CPI')
print(X_feateng['Store_group_CPI'].value_counts())

print('###')

print('Sore groups based on weekly sales and unemployment rate')
print(X_feateng['Store_group_unemp'].value_counts())


Store groups based on weekly sales and CPI
lowsales_highCPI     40
highsales_highCPI    34
highsales_lowCPI     25
lowsales_lowCPI      21
Name: Store_group_CPI, dtype: int64
###
Sore groups based on weekly sales and unemployment rate
highsales_highunemp    37
lowsales_lowunemp      35
highsales_lowunemp     24
lowsales_highunemp     21
Name: Store_group_unemp, dtype: int64


### Convert quarter variable to string

In [106]:
X_feateng['quarter_str'] = X_feateng['quarter'].map(quarter_dic)
# Check conversion
X_feateng.quarter_str.value_counts()

q2    35
q3    28
q1    27
q4    23
Name: quarter_str, dtype: int64

### Convert week variable to string

In [107]:
X_feateng.weekofyear = ['w'+str(int(week)) if ~np.isnan(week) else None for week in X_feateng.weekofyear.tolist()]
print(X_feateng.weekofyear.tolist()[:5])

['w7', 'w12', None, 'w21', 'w21']


### Categorize temperature
Categorize certain numerical variables to see if we can improve performance of model 

In [108]:
# TEMPERATURE
 
# Temperature distribution
print(X_feateng['Temperature'].describe())

# Categorize using bins function
X_feateng['Temperature_group'] = temp_bin_fn(data=X_feateng, temp_col='Temperature', temp_low=7, temp_high=25)

print('Data shape after feat eng:', X_feateng.shape)
print('Value counts:')
X_feateng.Temperature_group.value_counts()

count    117.000000
mean      15.781054
std       10.259300
min       -7.338889
25%        7.122222
50%       16.550000
75%       24.188889
max       33.138889
Name: Temperature, dtype: float64
Data shape after feat eng: (131, 14)
Value counts:


mean_temp    63
low_temp     28
high_temp    26
Name: Temperature_group, dtype: int64

In [109]:
X_feateng.shape

(131, 14)

### Holiday Flag: update flag to correspond to week near a holiday  

Since weekly sales are reported every Friday, the probability of having a holiday fall on that day is low. Create a variable *weekofyear_holiday* to indicate if the week of the sales corresponds to a week where there is an important holiday or (religious) event.   

Events collected from: [timeanddate](https://www.timeanddate.com/holidays/us/2011)

In [110]:
# Import dataframe indicating if there is a holiday on given week of the year
usa_hols_df.head()


,year,weekofyear,weekofyear_holiday
0,y1,wwww53,1
1,y1,wwww3,1
2,y1,wwww5,1
3,y1,wwww6,1
4,y1,wwww13,1


In [111]:
# Merge usahols to Xfeateng

# Convert weekofyear to str for merging
usa_hols_df.weekofyear = ['w'+str(week) for week in usa_hols_df.weekofyear.tolist() ]

# merge
X_feateng = X_feateng.merge(usa_hols_df, how='left', on=['year','weekofyear'])

# Update weekofyear_holiday with Holiday_Flag
vals_update = (np.isnan(X_feateng['weekofyear_holiday'])) & (X_feateng['Holiday_Flag']==0) 

X_feateng.loc[vals_update, 'weekofyear_holiday'] = 0
X_feateng.head()

,quarter,year,weekofyear,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Store_str,Store_group_CPI,Store_group_unemp,quarter_str,Temperature_group,weekofyear_holiday
0,1.0,y2,w7,1572117.54,NaN,15.338889,3.045,214.777523,6.858,6.0,highsales_highCPI,highsales_lowunemp,q1,mean_temp,NaN
1,1.0,y2,w12,1807545.43,0.0,5.766667,3.435,128.616064,7.470,13.0,highsales_lowCPI,highsales_highunemp,q1,low_temp,0.0
2,NaN,NaN,None,1244390.03,0.0,29.205556,NaN,214.556497,7.346,11.0,lowsales_highCPI,lowsales_lowunemp,NaN,high_temp,0.0
3,2.0,y1,w21,1644470.66,0.0,26.050000,2.759,212.412888,7.092,6.0,highsales_highCPI,highsales_lowunemp,q2,high_temp,0.0
4,2.0,y1,w21,1857533.70,0.0,NaN,2.756,126.160226,7.896,4.0,highsales_lowCPI,highsales_highunemp,q2,NaN,0.0


In [112]:
X_feateng.shape

(131, 15)

In [113]:
# Compare holiday_flag vs new feature weekofyear_holiday: number of positive values
X_feateng.Holiday_Flag.value_counts()

0.0    112
1.0      8
Name: Holiday_Flag, dtype: int64

In [114]:
X_feateng.weekofyear_holiday.value_counts()

0.0    112
Name: weekofyear_holiday, dtype: int64

### Final dataframe with engineered features

In [115]:
X_feateng.describe(include='all')

,quarter,year,weekofyear,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Store_str,Store_group_CPI,Store_group_unemp,quarter_str,Temperature_group,weekofyear_holiday
count,113.000000,113,113,1.310000e+02,120.000000,117.000000,119.000000,120.000000,117.000000,131,120,117,113,117,112.0
unique,NaN,3,45,NaN,NaN,NaN,NaN,NaN,NaN,19,4,4,4,3,NaN
top,NaN,y1,w25,NaN,NaN,NaN,NaN,NaN,NaN,3.0,lowsales_highCPI,highsales_highunemp,q2,mean_temp,NaN
freq,NaN,49,6,NaN,NaN,NaN,NaN,NaN,NaN,12,40,37,35,63,NaN
mean,2.415929,NaN,NaN,1.257990e+06,0.066667,15.781054,3.302908,180.175755,7.399427,NaN,NaN,NaN,NaN,NaN,0.0
std,1.066751,NaN,NaN,6.577463e+05,0.250490,10.259300,0.475435,39.723167,0.994117,NaN,NaN,NaN,NaN,NaN,0.0
min,1.000000,NaN,NaN,2.689290e+05,0.000000,-7.338889,2.514000,126.111903,5.143000,NaN,NaN,NaN,NaN,NaN,0.0
25%,2.000000,NaN,NaN,5.842439e+05,0.000000,7.122222,2.824000,132.579257,6.664000,NaN,NaN,NaN,NaN,NaN,0.0
50%,2.000000,NaN,NaN,1.366396e+06,0.000000,16.550000,3.435000,197.655672,7.368000,NaN,NaN,NaN,NaN,NaN,0.0
75%,3.000000,NaN,NaN,1.809576e+06,0.000000,24.188889,3.708500,214.904838,8.099000,NaN,NaN,NaN,NaN,NaN,0.0


In [116]:
X_feateng.shape

(131, 15)

## 3. Export processed variables for model training  

In [117]:
# Export target variable
print('Target variable shape:',Y.shape)
Y.to_csv('data/processed/Walmart_Store_sales-targetvar.csv', header=False, index=False)

Target variable shape: (131,)


### Basic features

In [118]:
# List of basic explanatory variables
basic_vars_ls = ["Store_str", "quarter", "year", "Fuel_Price", "weekofyear", "Holiday_Flag", "Temperature", "CPI", "Unemployment"]

# Subset X
X = X_raw.copy()
X = X[basic_vars_ls]

# Verify shape and export
print(X.shape)
display(X.head(3))
X.to_csv('data/processed/Walmart_Store_sales-expvar-basic.csv', index=False)

(131, 9)


,Store_str,quarter,year,Fuel_Price,weekofyear,Holiday_Flag,Temperature,CPI,Unemployment
0,6.0,1.0,y2,3.045,7.0,NaN,15.338889,214.777523,6.858
1,13.0,1.0,y2,3.435,12.0,0.0,5.766667,128.616064,7.470
2,11.0,NaN,NaN,NaN,NaN,0.0,29.205556,214.556497,7.346


### Engineered features

In [119]:
# List of engineered explanatory variables

eng_vars_ls = [ "quarter_str", "year", "Fuel_Price", "Temperature_group", "Store_group_CPI", "Store_group_unemp", "weekofyear_holiday"]

# Subset X
X_feateng_subset = X_feateng.copy()
X_feateng_subset = X_feateng_subset[eng_vars_ls]


# Verify shape and export
print(X_feateng_subset.shape)
display(X_feateng_subset.head(3))
X_feateng_subset.to_csv('data/processed/Walmart_Store_sales-expvar-feateng.csv', index=False)

(131, 7)


,quarter_str,year,Fuel_Price,Temperature_group,Store_group_CPI,Store_group_unemp,weekofyear_holiday
0,q1,y2,3.045,mean_temp,highsales_highCPI,highsales_lowunemp,NaN
1,q1,y2,3.435,low_temp,highsales_lowCPI,highsales_highunemp,0.0
2,NaN,NaN,NaN,high_temp,lowsales_highCPI,lowsales_lowunemp,0.0


## 4. Data processing summary

**Taget variable** : Y = *Weekly_Sales* (based on distribution no need to log values; no outliers)  

**Basic explanatory variables** : *quarter*, *year*, *weekofyear*, *Holiday_Flag*, *Temperature*, *CPI*, *Unemployment*

**Unique variables**: *Store_str*  

**Outlier samples**: Store 12

**Number of samples** : 131 (136 if no outlier removed)  

**Feature Engineering / Variable processing**:  
- *Store_group* : defined according to weekly  CPI and/or Unemployment (*Store_group_CPI* ; *Store_group_unemp*)
- *quarter_str* : categorize quarter  
- *Temperature_group* : categorize temperature in 3 groups based on min and max temperature  
- *weekofyear_holiday* : indicate if there is a holiday happening for that week of sales  